In [1]:
# Main notebook for investigating the parametersspace of vaccine vs cumulative incidence, from kommune-data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

from matplotlib.colors import LinearSegmentedColormap

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 


In [3]:

latestdir = list(os.walk(path_data))[-1][0]      
latestDate = pd.to_datetime(latestdir[-10:])  
         
latestDash = rootdir_data + f'\\ssi_dashboard\\SSI_dashboard_{latestDate:%Y-%m-%d}\\'
latestVacc = rootdir_data + f'\\ssi_vacc\\SSI_vacc_{latestDate:%Y-%m-%d}\\'
kortdf = pd.read_csv(latestDash+'Kommunalt_DB\\11_kort_pr_kommune.csv',delimiter = ';',dtype=str,encoding='latin1')
countdf = pd.read_csv(latestDash+'Kommunalt_DB\\17_tilfaelde_fnkt_alder_kommuner.csv',delimiter = ';',dtype=str,encoding='latin1')

vaccdf = pd.read_csv(latestVacc+'Vaccine_DB\\Vaccinationsdaekning_kommune.csv',delimiter = ';',dtype=str,encoding='latin1')

vaccdf['Antal borgere'] = pd.to_numeric(vaccdf['Antal borgere'])
vaccdf['Antal første vacc.'] = pd.to_numeric(vaccdf['Antal første vacc.'])
vaccdf['Antal faerdigvacc.'] = pd.to_numeric(vaccdf['Antal faerdigvacc.'])
vaccdf['Vacc.dækning foerste vacc. (%)'] = pd.to_numeric(vaccdf['Vacc.dækning foerste vacc. (%)'])
vaccdf['Vacc.dækning faerdigvacc. (%)'] = pd.to_numeric(vaccdf['Vacc.dækning faerdigvacc. (%)'])




sogndf = pd.read_csv(latestVacc+'Vaccination_sogne.csv',encoding='latin1')
sogndf['Inviterede_i_sognet'] = pd.to_numeric(sogndf['Inviterede_i_sognet'])
sogndf['Antal_inviteret_ikke_booket'] = pd.to_numeric(sogndf['Antal_inviteret_ikke_booket'],errors='coerce')

In [7]:
vaccdf.columns
# sogndf

,Kommune,Kommunenavn,Bekræftede tilfælde de seneste 7 dage,Testede de seneste 7 dage,Positivprocent 7 dage
0,101,København,4398,98293,"4,47"
1,147,Frederiksberg,678,14351,"4,72"
2,151,Ballerup,228,8593,"2,65"
3,153,Brøndby,264,5589,"4,72"
4,155,Dragør,78,2737,"2,85"
5,157,Gentofte,443,11559,"3,83"
6,159,Gladsaxe,368,10393,"3,54"
7,161,Glostrup,117,3429,"3,41"
8,163,Herlev,251,5588,"4,49"
9,165,Albertslund,124,4350,"2,85"


# Example

In [5]:
# curKommune = 'Ishøj'
largestKommuner = vaccdf.sort_values('Antal borgere',ascending=False)['Kommunenavn'].values[0:10]
# largestKommuner = vaccdf.sort_values('Antal borgere',ascending=True)['Kommunenavn'].values[0:10]
lowestCoverage = vaccdf.sort_values('Vacc.dækning faerdigvacc. (%)')['Kommunenavn'].values[0:10]
# lowestCoverage = vaccdf.sort_values('Vacc.dækning faerdigvacc. (%)',ascending=False)['Kommunenavn'].values[0:10]

kommuneList = largestKommuner
# kommuneList = ['Ishøj','Albertslund','Glostrup','København']
kommuneList = lowestCoverage

for curKommune in kommuneList:
    
    print('-----------------------------------------------')
    print(curKommune)
    if (curKommune == 'Århus'):
        curSogndf = sogndf[sogndf.Kommune == 'Aarhus']
    elif (curKommune == 'Høje Tåstrup'):
        curSogndf = sogndf[sogndf.Kommune == 'Høje-Taastrup']
    else:
        curSogndf = sogndf[sogndf.Kommune == curKommune]
    curVaccdf = vaccdf[vaccdf.Kommunenavn == curKommune]
    # display(curSogndf)
    # display(curVaccdf)

    numBorger = int(curVaccdf['Antal borgere'].values[0])
    numInvite = int(curSogndf['Inviterede_i_sognet'].sum())
    numInvite_NotBooked = int(curSogndf['Antal_inviteret_ikke_booket'].sum()) 

    maxPossible = numInvite/numBorger
    print(f'Højest mulige vaccinationsdækning: {100*maxPossible:2.1f}%')
    curPossible = (numInvite-numInvite_NotBooked)/numBorger
    print(f'Forventet vaccinationsdækning: {100*curPossible:2.1f}%')
    vaccAccRate = (numInvite-numInvite_NotBooked)/numInvite
    print(f'Andel der har taget imod vaccinetilbud: {100*vaccAccRate:2.1f}%')

-----------------------------------------------
Ishøj
Højest mulige vaccinationsdækning: 85.4%
Forventet vaccinationsdækning: 64.6%
Andel der har taget imod vaccinetilbud: 75.6%
-----------------------------------------------
Brøndby
Højest mulige vaccinationsdækning: 86.1%
Forventet vaccinationsdækning: 68.0%
Andel der har taget imod vaccinetilbud: 79.0%
-----------------------------------------------
Albertslund
Højest mulige vaccinationsdækning: 86.0%
Forventet vaccinationsdækning: 69.3%
Andel der har taget imod vaccinetilbud: 80.6%
-----------------------------------------------
Høje Tåstrup
Højest mulige vaccinationsdækning: 85.7%
Forventet vaccinationsdækning: 69.1%
Andel der har taget imod vaccinetilbud: 80.6%
-----------------------------------------------
Vallensbæk
Højest mulige vaccinationsdækning: 83.2%
Forventet vaccinationsdækning: 70.3%
Andel der har taget imod vaccinetilbud: 84.4%
-----------------------------------------------
Rødovre
Højest mulige vaccinationsdækning:

In [6]:
# sogndf.Kommune.unique()
# sogndf[sogndf.Kommune == 'Høje-Taastrup']

numBorger = int(vaccdf['Antal borgere'].sum())
numInvite = int(sogndf['Inviterede_i_sognet'].sum())
numInvite_NotBooked = int(sogndf['Antal_inviteret_ikke_booket'].sum()) 

print('Hele landet: ')
maxPossible = numInvite/numBorger
print(f'Højest mulige vaccinationsdækning: {100*maxPossible:2.1f}%')
curPossible = (numInvite-numInvite_NotBooked)/numBorger
print(f'Forventet vaccinationsdækning: {100*curPossible:2.1f}%')
vaccAccRate = (numInvite-numInvite_NotBooked)/numInvite
print(f'Andel der har taget imod vaccinetilbud: {100*vaccAccRate:2.1f}%')

# numBorger
# numInvite
# sogndf['Inviterede_i_sognet'].sum()
# vaccdf['Antal borgere'].values.sum()

Hele landet: 
Højest mulige vaccinationsdækning: 87.3%
Forventet vaccinationsdækning: 76.8%
Andel der har taget imod vaccinetilbud: 88.0%


In [7]:
# testdf = vaccdf.copy()
# testdf.sort_values('Vacc.dækning faerdigvacc. (%)')['Kommunenavn'].values[0:10]
# kortdf 

curKommune = 'Ishøj'

curSogndf = sogndf[sogndf.Kommune == curKommune]
curVaccdf = vaccdf[vaccdf.Kommunenavn == curKommune]
# display(curSogndf)
# display(curVaccdf)

numBorger = int(curVaccdf['Antal borgere'].values[0])
numInvite = int(curSogndf['Inviterede_i_sognet'].sum())
numInvite_NotBooked = int(curSogndf['Antal_inviteret_ikke_booket'].sum()) 

maxPossible = numInvite/numBorger
print(f'Højest mulige vaccinationsdækning: {100*maxPossible:2.1f}%')
curPossible = (numInvite-numInvite_NotBooked)/numBorger
print(f'Forventet vaccinationsdækning: {100*curPossible:2.1f}%')
vaccAccRate = (numInvite-numInvite_NotBooked)/numInvite
print(f'Andel der har taget imod vaccinetilbud: {100*vaccAccRate:2.1f}%')


print(numBorger)
totCases = pd.to_numeric(countdf[countdf.Kommune == '183']['Bekræftede tilfælde']).sum()
print(totCases/numBorger)

Højest mulige vaccinationsdækning: 85.4%
Forventet vaccinationsdækning: 64.6%
Andel der har taget imod vaccinetilbud: 75.6%
23175
0.18110032362459547


In [8]:
countdf[countdf.Kommune == '183']

,Kommune,Aldersgruppe,Bekræftede tilfælde,Dagsdato
16,183,0-2,77,2021-11-12
113,183,3-5,144,2021-11-12
212,183,6-11,444,2021-11-12
311,183,12-15,312,2021-11-12
411,183,16-19,294,2021-11-12
513,183,20-39,1423,2021-11-12
618,183,40-64,1238,2021-11-12
720,183,65-79,215,2021-11-12
820,183,80+,50,2021-11-12
